In [86]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [87]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [88]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [89]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[57, 58, 67, 73,  1, 73, 61, 54],
        [ 1, 72, 73, 71, 54, 67, 60, 58],
        [58,  9,  1, 59, 54, 71,  1, 74],
        [ 0, 61, 54, 57,  1, 74, 67, 73]], device='cuda:0')
targets:
tensor([[58, 67, 73,  1, 73, 61, 54, 73],
        [72, 73, 71, 54, 67, 60, 58,  1],
        [ 9,  1, 59, 54, 71,  1, 74, 67],
        [61, 54, 57,  1, 74, 67, 73, 62]], device='cuda:0')


In [90]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [91]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [92]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


My&,Pg_1lKfG)Rh"i0_Pw-pxH6o QQQweh5R'60oFRe-ZZ
gU.Z5t38-_sUyQKEVwW&)wAxU.Hz&jhKdYmY:'b0isz--dg!﻿"jl1!aTu'8FyF&ozhr4'9u8?1.DFt6H;:,e!!KykIPsza_
h2GxhaE36e)wDh4,Yyt80Id*UpUak﻿t0:FxkGD,
!zCytFfFeZP.*j,V:,kopa_ZT?UzU-*t-Q2:J7WQ?n,LV﻿[L&8]M!DOkrRZPli2CH
w3KtKkj7eZ?-I﻿eYia(w1mh5kn"ey,k*L3kBODQY-kUm8YY.dBvC't
46i73Gz]27co(IMzepnpbON34Us'vR'4;:NmYyY7a2Z.)4qM0y_uurnWz5WvYlD]Txni'!FrzZH61eTPyg﻿OmV6sY:XsvXkIH5OjL.cfXjt4GW7cJITk&vt5O;9Q"T8Y)pNe:rgJM m5Y[
o[&BdfpDqMBcJ9Uj M-kd4Fd7Qbp](lfR[WBHVar72]uxqQ([Y-)0


In [94]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f'step: {iter}, loss: {losses}')

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

NameError: name 'losses' is not defined

In [77]:
context= torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


YHC:As4﻿[w4?S;RJRH7-)CbgAJCh?A"EK4S"X9Sh;[*w"J_wBc&w8h5S"mFkZR.KK0mh"'jh
JLvtkMCttc0-IXsgiese0L8y2TxS!W1
U2bZLT0t&.lN8;YJ(aep A1L9K""3F [r5.Tc&W";Rxt
JNuxsJS*5hujX(lMwR&q 6ChpFtg_5ad3Om8Og_d240X:wMevULe'_l,(ta!l?DA,K0Q?HMvkMy0I))i0YT_5_13q5ZF-'HUw&eLWC?!c091*IhjUIizLnIL9H8"jori*YT5FKrY)g340qzlzh'4G;VY05ScCn*33qsm9;O4*l8Z!5o"(Ll.3A-xP;a5L'704xsnX2T'j!Qe2njGRX 'r"!ae﻿&)g3q'Iw&)OABC1L_﻿IFY;(XL23o"m EnzyB7K4S*[Ru.(jW5S*h)DAnIvscE*ngnIfBCTV0'[1Lp(9_dJ2Exb74:Ce1w]YT
Ib-xzK?mmzTzYpmRfdi*4S*kqyisd2d2E3[
